# Scalable Vector Data Analysis

You can access this notebook (in a Docker image) on this [GitHub repo](https://github.com/HamedAlemo/vector-data-tutorial).

In this lecture, we are going to use `Dask-GeoPandas` package to read a large vector dataset from [Source Cooperative](https://source.coop). Then use Dask parrallel computation to apply a spatial join operation to two geospatial DataFrames. 

Our target dataset is the `Google-Microsoft Open Buildings - combined by VIDA` dataset hosted on [Source Cooperative](https://source.coop/repositories/vida/google-microsoft-open-buildings/description). This is a combined version of the [Google](https://sites.research.google/open-buildings/) and [Microsoft](https://planetarycomputer.microsoft.com/dataset/ms-buildings) Open Buildings datasets and it has files in GeoParquet format hosted on AWS S3 bucket. Read the dataset description to familiarize yourself with the dataset and its structure. 

GeoParquet is a relatively new and open data format for column-oriented geospatial data. This format is build on the existing Apache Parquet format which is a very powerful format replacing CSV. You can check the specification [here](https://geoparquet.org/), and read more about the format on [this website](https://geoparquet.org/). In short, this format is interoperable, compressed and designed to work with large scale datasets. 

## Source Cooperative

Source Cooperative is a neutral, non-profit data-sharing utility that allows trusted organizations to share data without purchasing a data portal SaaS subscription or managing infrastructure. Source Cooperative is managed by Radiant Earth, and hosts 10s of datasets on its repository. 

In order to access/download data, you need to create a free account on Source Cooperative. Click on Sign in/up at the top of the page [here](https://source.coop/), and follow the steps to create an account. Make sure to use a non-Clark email to keep your account active after your graduation. 

### AWS Set up

All data on Source Cooperative, are hosted on AWS S3 bucket. In order to access them, you need  credentials that you can generate on Source Cooperative website. Atfer logging in, click on your name at the top right corner, and then click on your username. Then navigate to "Manage" page on the left side. At the bottom of this page you will find a section called "API Keys". If no key has been generated before, generate a new one and then copy the values for each of the following keys, and paste them in the following cell: 

In [1]:
AWS_ACCESS_KEY_ID = "<YOUR ACCESS KEY>"
AWS_SECRET_ACCESS_KEY = "<YOUR SECRET ACCESS KEY>"

Next, you need to create a s3 client from `boto3` library using your Source Cooperative credentials:

In [2]:
import boto3
s3_client = boto3.client('s3',
                         aws_access_key_id = AWS_ACCESS_KEY_ID, 
                         aws_secret_access_key = AWS_SECRET_ACCESS_KEY,
                         endpoint_url='https://data.source.coop'
                        )

## Download and Load Buildings Footprint Data into Dask-GeoPandas

First, we start a new Dask cluster:

In [3]:
from dask.distributed import Client, LocalCluster
cluster = LocalCluster()
client = Client(cluster)
print(client.dashboard_link)

http://127.0.0.1:8787/status


2024-11-12 04:21:51,092 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle fdfed24b60e0cb5cbcfe34ef15878a42 initialized by task ('shuffle-transfer-fdfed24b60e0cb5cbcfe34ef15878a42', 4) executed on worker tcp://127.0.0.1:39441
2024-11-12 04:22:07,442 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle fdfed24b60e0cb5cbcfe34ef15878a42 deactivated due to stimulus 'task-finished-1731385327.4416423'


In this lecture, we are going to access the data from a specific country. As noted in the dataset description, you need the 3-letter country ISO name to access the corresponding file. You can look up the ISO name for your country of choice [here](https://www.iso.org/obp/ui/#search).

You need to enter the ISO name and the EPSG corresponding to the UTM zone of your country of choice in the following:

In [4]:
#Rwanda
country_code = "RWA"
country_epsg = 32736

Define a path to download the data:

In [5]:
local_path = "./data/"

Let's import our function from `utils` module and run it. This function uses Dask-GeoPandas to lazy load the data from GeoParquet format into memory. 

In [6]:
from utils import get_google_microsoft_bldgs

The following cell downloads the geoparquet file from s3 bucket, and loads it into `Dask-GeoPandas` `GeoDataFrame`. We are using a default value of 256M for the blocksize in Dask. If you run into memory issue in the rest of the notebook, lower the blocksize and re-run the following cell. 

In [7]:
bldg_ddf = get_google_microsoft_bldgs(country_code, s3_client, local_path, blocksize = "256M")

File already exists locally. No download needed.


In [8]:
bldg_ddf

,boundary_id,bf_source,confidence,area_in_meters,s2_id,country_iso,geohash,geometry,bbox
npartitions=5,,,,,,,,,
,int64,object,float64,float64,int64,object,object,geometry,object
,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...


## Read Adminsitrative Boundaries Dataset

We are also interested to load the adminsitrative boundaries dataset for our country of choice, and assign each building an administrative unit (Parish) name. 

You can download each countries administrative unit json files on GDAM [website](https://gadm.org/download_country.html). Each country has different number of levels for their administrative units (and not all are available on GDAM website). 

Check your country of choice, and find what is the highest level of administrative boundaries that is available. 

In the following, we are interested in level 4 data, and the following function will download it. 

In [9]:
from utils import get_gdam_json

In [10]:
boundaries = get_gdam_json(country_code = country_code, admin_level=1)

ERROR 1: PROJ: proj_create_from_database: Open of /opt/conda/envs/vector_tutorial/share/proj failed


In [11]:
boundaries.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [12]:
boundaries.head()

,GID_1,GID_0,COUNTRY,Parish,VARNAME_1,NL_NAME_1,TYPE_1,ENGTYPE_1,CC_1,HASC_1,ISO_1,geometry
0,RWA.1_1,RWA,Rwanda,Amajyaruguru,NorthernProvince|ProvinceduNo,NA,Province,Province,4,RW.NO,NA,"MULTIPOLYGON (((29.9924 -1.9076, 29.9828 -1.91..."
1,RWA.2_1,RWA,Rwanda,Amajyepfo,SouthernProvince|ProvinceduSu,NA,Province,Province,2,RW.SU,NA,"MULTIPOLYGON (((29.5407 -2.8291, 29.5381 -2.82..."
2,RWA.3_1,RWA,Rwanda,Iburasirazuba,EasternProvince|Provincedel'E,NA,Province,Province,5,RW.ES,NA,"MULTIPOLYGON (((30.6679 -2.4039, 30.664 -2.405..."
3,RWA.4_1,RWA,Rwanda,Iburengerazuba,WesternProvince|Provincedel'O,NA,Province,Province,3,RW.OU,NA,"MULTIPOLYGON (((29.0355 -2.7375, 29.0347 -2.73..."
4,RWA.5_1,RWA,Rwanda,UmujyiwaKigali,KigaliCity|VilledeKigali,NA,Province,Province,1,RW.KV,NA,"MULTIPOLYGON (((30.0161 -2.0755, 30.0164 -2.07..."


## Spatial Join

Now, we will use the spatial join to add the Parish name (`Parish` column in `boundaries`) to `bldg_ddf`:

In [13]:
bldg_ddf_w_boundaries = bldg_ddf.sjoin(boundaries, how="inner", predicate="intersects")

In [14]:
bldg_ddf_w_boundaries

,boundary_id,bf_source,confidence,area_in_meters,s2_id,country_iso,geohash,geometry,bbox,index_right,GID_1,GID_0,COUNTRY,Parish,VARNAME_1,NL_NAME_1,TYPE_1,ENGTYPE_1,CC_1,HASC_1,ISO_1
npartitions=5,,,,,,,,,,,,,,,,,,,,,
,int64,object,float64,float64,int64,object,object,geometry,object,int64,string,string,string,string,string,string,string,string,string,string,string
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [15]:
buildigs_per_parish = bldg_ddf_w_boundaries["Parish"].value_counts().compute()

In [16]:
buildigs_per_parish

Parish
Amajyaruguru      1083456
Amajyepfo         1471584
Iburengerazuba    1273120
Iburasirazuba     1703538
UmujyiwaKigali     770553
Name: count, dtype: int64[pyarrow]

In [17]:
print(f"Total number of buildings is {buildigs_per_parish.sum()}")

Total number of buildings is 6302251


### Exercise 1: Plot the number of buildings per Parish on the map

### Exercise 2: Calculate percentage of the area of each Parish that is covered by buildings